In [1]:
import numpy as np
import pandas as pd
import ffmpeg
from tqdm import tqdm

In [2]:
lec = 1
transcript = pd.read_json(
	f'Data/Text/Lec{lec}.json',
	orient='records',
	dtype={'a': np.uint8}, # type: ignore
)
transcript.drop(columns=['i','t'], inplace=True)
transcript[['s','e']] /= 1000
# transcript

In [8]:
def join_sentence(df:pd.DataFrame) -> pd.Series:
	series = df.iloc[0]
	series['w'] = ' '.join(df['w'])
	series['e'] = df.iloc[-1]['e']
	series['a'] = df['a'].min()
	return series

sentences = transcript.groupby(
	transcript['w'].str.endswith('.').shift(fill_value=False).cumsum(),
	as_index=True,
).apply(join_sentence)

sentences.index.name = 'i'
sentences = sentences[sentences['a'] >= 70]
sentences = sentences[sentences['w'].str.count(' ') >= 3]
sentences.reset_index(inplace=True)
# # sentences.to_feather(f'Data/Text/Lec{lec}.feather') # save?
# sentences

0.70595690747782


In [4]:
# mp4 (AAC) -> wav (PCM)
ffmpeg.input(f'Data/Video/Lec{lec}.mp4').output(f'Data/Audio/Lec{lec}.wav').overwrite_output().run()

# clips from the whole wav
for thread in tqdm([
	ffmpeg.input(f'Data/Audio/Lec{lec}.wav')
		.output(f'Data/Audio/Lec{lec}-{t.i}.wav', ss=t.s, to=t.e)
		.overwrite_output()
		.run_async()
	for t in tqdm(sentences.itertuples(), total=len(sentences))
]): thread.wait() # wait until all is done

100%|██████████| 454/454 [00:15<00:00, 28.52it/s] 


In [5]:
# from scipy.io import wavfile
# wavfile.read('Data/Audio/Lec1-1.wav')